In [5]:
import autograd.numpy as np
import pyipopt
from autograd import grad, hessian, jacobian

import matplotlib.pyplot as plt
from counterfactual_functions import *
from helper import *



username='boraozaltun'

data_dict = load_obj('/Users/'+username+'/Dropbox (MIT)/Data/Trade/general_equilibrium_gravity/final_data/data_20200219.pickle')

reg_2_num = load_obj('/Users/'+username+'/Dropbox (MIT)/Data/Trade/general_equilibrium_gravity/GTAP/mapping/reg_2_num.pickle')
comm_2_num = load_obj('/Users/'+username+'/Dropbox (MIT)/Data/Trade/general_equilibrium_gravity/GTAP/mapping/comm_2_num.pickle')


def eval_jac_g(X, flag, user_data=None):
    """
    Evaluate the sparse Jacobian of constraint functions g.
    @param X: parameter values
    @param flag: this asks for the sparsity structure
    """
    print('eval_jac_g')

    if flag:
        temp = np.ones((X.shape[0], X.shape[0]))
        rows, cols = np.nonzero(temp)
        #rows = np.linspace(0, len(X) -1, len(X)).astype(int)
        #cols = np.linspace(0, len(X) -1, len(X)).astype(int)
        return (rows, cols)
    else:

        return jacobian(eval_g)(X)


n = data_dict['n']
g = data_dict['g']
k = data_dict['k']

data_dict['R_hat'] = np.ones((n, g))
X_0 = np.ones(n*g + n)*1.0

eval_g = lambda x: reduced_counterfactual(x, data_dict)
eval_f = lambda x: np.linalg.norm(reduced_counterfactual(x, data_dict))

nvar = X_0.shape[0]
x_L = np.zeros((nvar))
x_U = np.ones((nvar)) * 10000.0

ncon = nvar

g_L = np.zeros((nvar))*1.0
g_U = np.zeros((nvar))*1.0

eval_grad_f = grad(eval_f)

nnzj = nvar*nvar
nnzh =  int((nvar*(nvar+1))/2)


In [6]:

nlp = pyipopt.create(nvar, x_L, x_U, ncon, g_L, g_U, nnzj, nnzh, eval_f, eval_grad_f, eval_g, eval_jac_g)

print("Going to call solve")
print("x0 = {}".format(X_0))


Going to call solve
x0 = [1. 1. 1. ... 1. 1. 1.]


In [ ]:
x, zl, zu, constraint_multipliers, obj, status = nlp.solve(X_0)

print('x:', x)
print('zl:', zl)
print('zu:', zu)
print('constraint_multipliers:', constraint_multipliers)
print('obj:', obj)
print('status:', status)

save_obj(x, 'x_output.pickle')





eval_jac_g


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/autograd/numpy/numpy_vjps.py:444: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return lambda g: g[idxs]


eval_jac_g
